<center>

# $\textbf{Merge Files}$

<center>

### $\textbf{Code}$

In [ ]:
from pyspark.sql import SparkSession
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf "spark.executor.extraJavaOptions=-XX:ReservedCodeCacheSize=2048m" --conf "spark.driver.extraJavaOptions=-XX:ReservedCodeCacheSize=2048m" pyspark-shell'

In [ ]:
spark = SparkSession.builder.appName('MergeFiles').master("local").enableHiveSupport().getOrCreate()
spark

In [ ]:
# Set paths to the Parquet files
file_paths = ["FilesParquet/Covid.parquet", "FilesParquet/GDP.parquet", "FilesParquet/Inflation.parquet",
              "FilesParquet/Migration.parquet", "FilesParquet/Population.parquet", "FilesParquet/Tax.parquet", "FilesParquet/Unemployment.parquet"]

# Read Parquet files into DataFrames
dfs = [spark.read.parquet(file_path) for file_path in file_paths]

# Merge or outer join the DataFrames
merged_df = dfs[0]
for i, df in enumerate(dfs):
    # Generate new column names with a suffix indicating the iteration number
    renamed_columns = [(c, f"{c}_df{i+1}") for c in df.columns if c == "year"]
    # Rename columns in the DataFrame
    for old_name, new_name in renamed_columns:
        df = df.withColumnRenamed(old_name, new_name)
    # Update the DataFrame in the list
    dfs[i] = df

for df in dfs[1:]:
    merged_df = merged_df.join(df, on="country", how="outer")

In [ ]:
#Storing this dataframe in parquet
merged_df.write.mode("overwrite").parquet("FinalOuterParquet")
spark.read.parquet("FilesParquet/Covid").show()
spark.stop()